In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [2]:
import sys
print(sys.executable)

c:\Users\HP\anaconda3\envs\pose_env\python.exe


In [3]:
model_path = 'pose_landmarker_lite.task'

In [4]:
import os
import numpy as np
import cv2

In [ ]:
import os
import numpy as np
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE
)

dataset_path = "C:/Users/HP/Desktop/SEM 6/Minor project 2/Book-Recommendation-System-master/Yoga Pose Detector/TRAIN"
X = []
y = []

with PoseLandmarker.create_from_options(options) as landmarker:
    for label_folder in os.listdir(dataset_path):

        label_path = os.path.join(dataset_path, label_folder)
        if not os.path.isdir(label_path):
            continue

        images_folder = os.path.join(label_path, "Images")
        if not os.path.isdir(images_folder):
            continue

        for image_file in os.listdir(images_folder):
            if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            full_image_path = os.path.join(images_folder, image_file)

            test_img = cv2.imread(full_image_path)
            if test_img is None:
                print(f"Skipping invalid image: {full_image_path}")
                continue  # skip to next image


            image_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

            # Convert to MediaPipe Image object
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

            result = landmarker.detect(mp_image)

            if result.pose_landmarks and len(result.pose_landmarks) > 0:
                landmark_list = result.pose_landmarks[0]  # 1 person
                landmarks = []
                for lm in landmark_list:
                    landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
                X.append(landmarks)
                y.append(label_folder)

# Convert to arrays
X = np.array(X)
y = np.array(y)

print("Feature shape:", X.shape)
print("Labels shape:", y.shape)



Feature shape: (481, 132)
Labels shape: (481,)


In [8]:
print(X[0])

[ 0.35614049  0.46239594 -0.21268496  0.99991763  0.34732348  0.46317449
 -0.19650474  0.99988747  0.34739149  0.46180692 -0.19655327  0.99976844
  0.34748888  0.4604857  -0.19662991  0.99980897  0.34655154  0.4700532
 -0.20822865  0.99993563  0.3458735   0.47401676 -0.20836335  0.99991286
  0.34513178  0.47955954 -0.20835443  0.99991286  0.35056537  0.46947071
 -0.10418595  0.99981219  0.34690291  0.50305068 -0.15892996  0.9999187
  0.36656022  0.46230638 -0.17770208  0.99984217  0.3655684   0.4723323
 -0.19353841  0.99990666  0.40174121  0.43724573 -0.06370167  0.99998701
  0.3972159   0.56435937 -0.1158645   0.99996436  0.39970022  0.33423921
 -0.07111737  0.96530461  0.39857864  0.67586994 -0.15624148  0.98875779
  0.39453667  0.2389296  -0.15579689  0.95139754  0.38452351  0.78856832
 -0.22632419  0.98145998  0.39222777  0.21197483 -0.17848668  0.89467818
  0.38001657  0.8118248  -0.24293949  0.95548427  0.39079529  0.21093714
 -0.19959982  0.90009177  0.37805361  0.81118006 -0.26

In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # e.g., 'Tadasana' -> 0, 'Vrikshasana' -> 1
np.save('label_encoder_classes.npy', label_encoder.classes_)


In [10]:
for i, pose_name in enumerate(label_encoder.classes_):
    print(f"Encoded value {i} corresponds to: {pose_name}")

Encoded value 0 corresponds to: ArdhaChandrasana
Encoded value 1 corresponds to: BaddhaKonasana
Encoded value 2 corresponds to: Downward_dog
Encoded value 3 corresponds to: Natarajasana
Encoded value 4 corresponds to: Triangle
Encoded value 5 corresponds to: UtkataKonasana
Encoded value 6 corresponds to: Veerabhadrasana
Encoded value 7 corresponds to: Vrukshasana


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(132,)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)


Epoch 1/20


c:\Users\HP\anaconda3\envs\pose_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.2000 - loss: 2.0059 - val_accuracy: 0.3814 - val_loss: 1.8232
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4947 - loss: 1.7564 - val_accuracy: 0.5361 - val_loss: 1.5827
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5604 - loss: 1.5152 - val_accuracy: 0.6495 - val_loss: 1.3828
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6913 - loss: 1.2773 - val_accuracy: 0.7423 - val_loss: 1.2018
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7436 - loss: 1.1120 - val_accuracy: 0.7113 - val_loss: 1.0541
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7629 - loss: 0.9896 - val_accuracy: 0.7010 - val_loss: 0.9739
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7815 - loss: 0.8633 - val_accuracy: 0.7835 - val_loss: 0.8449
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8213 - loss: 0.7547 - val_accuracy: 0.8041 - val_loss: 0.74

In [13]:


# For neural net
loss, acc = model.evaluate(X_test, y_test)
print("Validation accuracy:", acc)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9097 - loss: 0.3377 
Validation accuracy: 0.9278350472450256


In [14]:
model.save("yoga_pose_model.keras")
#Saving the final trained model